In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

import nltk
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
import time

from Vocab import Vocab

import torch
torch.cuda.set_device(1)

print('import over')

import over


In [2]:
def batch_words2sentence(words_list):
    return [' '.join(words) for words in words_list]
def batch_tokens2words(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return: words_list corresponding to tokens
    return [[vocab.token2word[token] for token in tokens] for tokens in tokens_list]

def batch_tokens_remove_eos(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return pure tokens_list removed eos symbol
    result=[]
    for tokens in tokens_list:
        tokens_filtered=[]
        for token in tokens:
            if token == vocab.word2token['<eos>']:
#                 tokens_filtered.append(token)
                break
            else:
                tokens_filtered.append(token)
        result.append(tokens_filtered)
    return result

def batch_tokens_bleu(references, candidates, smooth_epsilon=0.001):
    ##    para: references and candidates are list[list] type
    ##    return: list of BLEU for every sample
    ##
    bleu_scores=[]
    for ref, candidate in zip(references, candidates):
        if min(len(ref), len(candidate))<4:
            bleu_scores.append(0)
        else:
            bleu_scores.append(sentence_bleu([ref], candidate, smoothing_function = SmoothingFunction(epsilon=smooth_epsilon).method1))
    return bleu_scores

with open('data_set/vocab.pk', 'rb') as f:
    vocab=pickle.load(f)
    
batch_tokens_bleu([[1,2,3,4,5,6]], [[2,3,1,4,5]])

[0.021744100219015735]

In [3]:
with open('./data_set/fusion_data_set/train_pseudo_simple_sents.pk', 'rb') as f:
    fusion_pseudo_train_set_inputs = pickle.load(f)
with open('./data_set/fusion_data_set/train_pseudo_simple_sent_lens.pk', 'rb') as f:
    fusion_pseudo_train_set_input_lens = pickle.load(f)
with open('./data_set/fusion_data_set/train_pseudo_labels.pk', 'rb') as f:
    fusion_pseudo_train_set_labels = pickle.load(f)
    
with open('./data_set/fusion_data_set/validation_pseudo_simple_sents.pk', 'rb') as f:
    fusion_pseudo_valid_set_inputs = pickle.load(f)
with open('./data_set/fusion_data_set/validation_pseudo_simple_sent_lens.pk', 'rb') as f:
    fusion_pseudo_valid_set_input_lens = pickle.load(f)
with open('./data_set/fusion_data_set/validation_pseudo_labels.pk', 'rb') as f:
    fusion_pseudo_valid_set_labels = pickle.load(f)
    
    


with open('./data_set/split_data_set/train_complex_sents.pk', 'rb') as f:
    split_train_set_inputs = pickle.load(f)
with open('./data_set/split_data_set/train_complex_sent_lens.pk', 'rb') as f:
    split_train_set_input_lens = pickle.load(f)
with open('./data_set/split_data_set/train_pseudo_labels.pk', 'rb') as f:
    split_pseudo_train_set_labels = pickle.load(f)
    
with open('./data_set/split_data_set/validation_complex_sents.pk', 'rb') as f:
    split_valid_set_inputs = pickle.load(f)
with open('./data_set/split_data_set/validation_complex_sent_lens.pk', 'rb') as f:
    split_valid_set_input_lens = pickle.load(f)
with open('./data_set/split_data_set/validation_labels.pk', 'rb') as f:
    split_pseudo_valid_set_labels = pickle.load(f)


In [4]:
# num=100000
# train_set_inputs_10w = train_set_inputs[:num]
# train_set_input_lens_10w = train_set_input_lens[:num]
# train_set_labels_10w = train_set_labels[:num]

# valid_set_inputs_10w = valid_set_inputs[:num]
# valid_set_input_lens_10w = valid_set_input_lens[:num]
# valid_set_labels_10w = valid_set_labels[:num]


# with open('./small_data_set/train_set_inputs_10w.pk', 'bw') as f:
#     pickle.dump(train_set_inputs_10w,f)
# with open('./small_data_set/train_set_input_lens_10w.pk', 'bw') as f:
#     pickle.dump(train_set_input_lens_10w ,f)
# with open('./small_data_set/train_set_labels_10w.pk', 'bw') as f:
#     pickle.dump(train_set_labels_10w ,f)
    
# with open('./small_data_set/valid_set_inputs_10w.pk', 'bw') as f:
#     pickle.dump(valid_set_inputs_10w ,f)
# with open('./small_data_set/valid_set_input_lens_10w.pk', 'bw') as f:
#     pickle.dump(valid_set_input_lens_10w ,f)
# with open('./small_data_set/valid_set_labels_10w.pk', 'bw') as f:
#     pickle.dump(valid_set_labels_10w ,f)

In [5]:
print(len(split_train_set_inputs), len(split_train_set_input_lens), len(split_pseudo_train_set_labels))
print(len(fusion_pseudo_train_set_inputs), len(fusion_pseudo_train_set_input_lens), len(fusion_pseudo_train_set_labels))
# for sent_len in valid_set_input_lens:
#     if sent_len<=2:
#         print('why')

989944 989944 989944
989944 989944 989944


In [6]:
class Encoder(nn.Module):
    def __init__(self, use_cuda, hidden_dim, input_dim, vocab):#, pre_train_weight, is_fix_word_vector = 1):
        super(Encoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.vocab = vocab
        
        self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
                                hidden_size= self.hidden_dim, 
                                bidirectional=True,
                                batch_first=True
                               )
        
        #embedding
        self.embed=nn.Embedding(len(self.vocab.word2token), input_dim)
        #loading pre trained word embedding
        with open('data_set/pre_trained_token_embedding.pk', 'rb') as f:
            pre_train_word_embedding = pickle.load(f)
            
        self.embed.weight.data.copy_(torch.FloatTensor(pre_train_word_embedding))
#         self.embed.weight.requires_grad = False
        
    def order(self, inputs, inputs_len):    #inputs: tensor, inputs_len: 1D tensor
        inputs_len, sort_ids = torch.sort(inputs_len, dim=0, descending=True)
        
        if self.use_cuda:
            inputs = inputs.index_select(0, Variable(sort_ids).cuda())
        else:
            inputs = inputs.index_select(0, Variable(sort_ids))
        
        _, true_order_ids = torch.sort(sort_ids, dim=0, descending=False)
        
        return inputs, inputs_len, true_order_ids
    #
    def forward(self, inputs, inputs_len):
        inputs = Variable(inputs)
        if self.use_cuda:
            inputs=inputs.cuda()
            
        inputs, sort_len, true_order_ids = self.order(inputs, inputs_len)

        in_vecs=self.embed(inputs)

        packed = rnn_utils.pack_padded_sequence(input=in_vecs, lengths=list(sort_len), batch_first =True)
        
        outputs, (hn,cn) = self.lstm(packed)
        outputs, sent_lens = rnn_utils.pad_packed_sequence(outputs)
        
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        outputs = outputs.transpose(0,1)  #transpose is necessary
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        
        #warnning: outputs, hn and cn have been sorted by sentences length so the order is wrong, now to sort them
        if self.use_cuda:
            outputs = outputs.index_select(0, Variable(true_order_ids).cuda())
        else:
            outputs = outputs.index_select(0, Variable(true_order_ids))
        
        hn = torch.cat((hn[0], hn[1]), dim=1)
        cn = torch.cat((cn[0], cn[1]), dim=1)
        #print('hn size and cn size: ', hn.size(), cn.size())
        
        if self.use_cuda:
            hn = hn.index_select(0, Variable(true_order_ids).cuda())
            cn = cn.index_select(0, Variable(true_order_ids).cuda())
        else:
            hn = hn.index_select(0, Variable(true_order_ids))
            cn = cn.index_select(0, Variable(true_order_ids))
            
        return outputs, (hn,cn)

In [7]:
def _inflate(tensor, times, dim):
    """
    Examples::
        >> a = torch.LongTensor([[1, 2], [3, 4]])
        >> a
        1   2
        3   4
        [torch.LongTensor of size 2x2]
        >> b = ._inflate(a, 2, dim=1)
        >> b
        1   2   1   2
        3   4   3   4
        [torch.LongTensor of size 2x4]
    """
    repeat_dims = [1] * tensor.dim()
    repeat_dims[dim] = times
    return tensor.repeat(*repeat_dims)

class Decoder(nn.Module):
    def __init__(self, use_cuda, encoder, hidden_dim, max_length=25):
        super(Decoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.hidden_dim=hidden_dim
        self.input_dim = encoder.input_dim
        self.max_length = max_length
        self.vocab = encoder.vocab
        self.weight = [1]*len(self.vocab.word2token)
        self.weight[self.vocab.word2token['<padding>']]=0
        #self.weight[self.vocab.word2token['<eos>']]=1.01
        #self.weight[self.vocab.word2token['<split>']]=1.01
        
        self.hidden_size = self.hidden_dim
        self.V = len(self.vocab.word2token)
        self.SOS = self.vocab.word2token['<sos>']
        self.EOS = self.vocab.word2token['<eos>']
        self.log_softmax = nn.LogSoftmax(dim=1)
        
        self.lstmcell = torch.nn.LSTMCell(input_size=self.input_dim, hidden_size=self.hidden_dim*2, bias=True)
        
        #embedding
        self.embed=encoder.embed# reference share
        #fcnn: projection for crossentroy loss
        self.fcnn = nn.Linear(in_features = self.hidden_dim*2, out_features = len(self.vocab.word2token))
        
        self.softmax = nn.Softmax(dim=1)
        self.cost_func = nn.CrossEntropyLoss(weight=torch.Tensor(self.weight), reduce=True)

        print('init lookup embedding matrix size: ', self.embed.weight.data.size())
        

    def forward(self, enc_outputs, sent_lens, h0_and_c0, labels, teaching_rate=0.6, is_train=1):
        labels = Variable(labels)
        if self.use_cuda:
            labels = labels.cuda()

        all_loss = 0
        predicts = []
        max_probs=[]
        batch_size = enc_outputs.size(dim = 0)
        final_hidden_states = h0_and_c0[0]

        for ii in range(self.max_length):
            if ii==0:
                zero_timestep_input = Variable(torch.LongTensor([self.vocab.word2token['<sos>']]*batch_size))
                if self.use_cuda:
                    zero_timestep_input = zero_timestep_input.cuda()
                    
                zero_timestep_input = self.embed(zero_timestep_input)#size: batch_size * self.input_dim

                last_timestep_hidden_state,cx = self.lstmcell(zero_timestep_input, h0_and_c0)
                #print('hn and cn sizes: ', last_timestep_hidden_state.size(), cx.size())
                
                last_timestep_output = self.fcnn(last_timestep_hidden_state)
                if is_train:
                    loss = self.cost_func(last_timestep_output, labels[:,0])
#                     t=torch.mean(loss)
                    all_loss+=loss
                
                #last_timestep_output = self.log_softmax(last_timestep_output)
                _, max_idxs = torch.max(last_timestep_output, dim=1)
#                 max_prob = self.cost_func(last_timestep_output, labels[:,0])
#                 max_probs.append(torch.unsqueeze(max_prob, dim=0))
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                #print('max_prob size: ', max_prob.size())
                #print('max_idxs size: ',max_idxs.size(), max_idxs)
                
            else:
                if is_train:
                    rand = random.random()
                    if rand<teaching_rate:
                        this_timestep_input = self.embed(labels[:,ii-1])#label teaching, lookup embedding
                    else:
                        this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                else:
                    this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                    
                last_timestep_hidden_state ,cx = self.lstmcell(this_timestep_input, (last_timestep_hidden_state,cx))
                last_timestep_output = self.fcnn(last_timestep_hidden_state)
                
                if is_train:
                    loss = self.cost_func(last_timestep_output, labels[:,ii])
#                     t=torch.mean(loss)
                    all_loss+=loss
#                 _, max_idxs = torch.max(last_timestep_output, dim=1)
#                 #print('max_idx size: ', max_idxs.size(), max_idxs)
#                 predicts.append(torch.unsqueeze(max_idxs, dim=0))
                #last_timestep_output = self.log_softmax(last_timestep_output)
                _, max_idxs = torch.max(last_timestep_output, dim=1)
#                 max_prob = self.cost_func(last_timestep_output, labels[:,ii])
#                 max_probs.append(torch.unsqueeze(max_prob, dim=0))
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                
        predicts = torch.cat(predicts, dim=0)
        predicts = torch.transpose(predicts, 0, 1)
    
        if is_train:  #training
#             all_loss = torch.cat(all_loss, dim=1)
#             all_loss = torch.mean(all_loss, dim=1)
#             loss = torch.mean(all_loss)
            loss = all_loss/self.max_length
            #print('loss size: ', loss.size())
            #torch.cuda.empty_cache()
            if self.use_cuda:
                return loss, predicts.data.cpu().tolist()
            else:
                return loss, predicts.data.tolist()
        else:   #testing
            if self.use_cuda:
                return predicts.data.cpu().tolist()
            else:
                return predicts.data.tolist()
#         if is_train:  #training
#             if self.use_cuda:
#                 return all_loss/(self.max_length+1), predicts.data.cpu().numpy()
#             else:
#                 return all_loss/(self.max_length+1), predicts.data.numpy()
#         else:   #testing
#             if self.use_cuda:
#                 return predicts.data.cpu().numpy()
#             else:
#                 return predicts.data.numpy()
    def loss_func_rl(self, logits):
        logprobs = self.log_softmax(logits)
        max_logprobs, max_ids = torch.max(logprobs, dim=1)
        weights = torch.FloatTensor(self.weight)
        index = max_ids.data
        if self.use_cuda:
            index = index.cpu()
        weights = Variable(torch.index_select(weights, 0, index), requires_grad=False)
        if self.use_cuda:
            weights = weights.cuda()
        return -max_logprobs*weights
    
    def token_sample(self, logits, batch_size, topk=100, e_greedy_rate=0.2, dec_seq_num=10):
        topk_logits, topk_idxs = torch.topk(logits, k=topk, dim=1)
        if self.use_cuda:
            ltopk_probs = self.softmax(topk_logits).data.cpu().tolist()
            ltopk_idxs = topk_idxs.data.cpu().tolist()
        else:
            ltopk_probs = self.softmax(topk_logits).data.tolist()
            ltopk_idxs = topk_idxs.data.tolist()
        batch_results=[]
        for probs, idxs in zip(ltopk_probs, ltopk_idxs):
            sum_probs = sum(probs)
            probs = [x/sum_probs for x in probs]
            results=[]
            for ii in range(dec_seq_num):
                result = np.random.choice(idxs, p=probs)
                if random.random()>e_greedy_rate:
                    results.append(int(result))
                else:
                    results.append(random.randint(0, len(self.vocab.word2token)-1))
            batch_results.append(results)
        return batch_results
    
    def train_using_rl_2(self, enc_outputs, sent_lens, h0_and_c0, labels, teaching_rate=0.6, is_train=1):
        #nll_loss = nn.NLLLoss(weight=torch.FloatTensor(self.weight).cuda(), reduce=False)
        
        # parameter labels must be python list type
        labels_tokens=labels
        labels = Variable(torch.LongTensor(labels))
        if self.use_cuda:
            labels = labels.cuda()
        #print('rl...')
        loss=0
        all_loss = 0
        losss=[]
        predicts = []
        max_probs=[]
        batch_size = enc_outputs.size(dim = 0)
        topk=20
        e_greedy_rate=0.3
        dec_seq_num=100
        vocab_size = len(self.vocab.word2token)
        index_bias = torch.LongTensor([vocab_size*ii for ii in range(batch_size)]).unsqueeze(dim=1)
        index_bias = index_bias.expand(batch_size, dec_seq_num).contiguous().view(-1)
        if self.use_cuda:
            index_bias = index_bias.cuda()
        for ii in range(self.max_length):
            if ii==0:
                zero_timestep_input = Variable(torch.LongTensor([self.vocab.word2token['<sos>']]*batch_size))
                if self.use_cuda:
                    zero_timestep_input = zero_timestep_input.cuda()
                    
                zero_timestep_input = self.embed(zero_timestep_input)#size: batch_size * self.input_dim

                last_timestep_hidden_state,cx = self.lstmcell(zero_timestep_input, h0_and_c0)
                #print('hn and cn sizes: ', last_timestep_hidden_state.size(), cx.size())
                
                last_timestep_output = self.fcnn(last_timestep_hidden_state)

                batch_ltokens = self.token_sample(logits=last_timestep_output, batch_size=batch_size, topk=topk, e_greedy_rate=e_greedy_rate, dec_seq_num=dec_seq_num)
                #print('batch_ltokens size: ', torch.LongTensor(batch_ltokens).size())
                batch_ltokens=torch.LongTensor(batch_ltokens).view(-1)
                if self.use_cuda:
                    batch_ltokens = batch_ltokens.cuda()
                predicts.append(batch_ltokens.view(-1,1))
                if is_train:
                    neg_log_prob = -self.log_softmax(last_timestep_output).view(-1)
                    #print('batch_ltokens and index_bias size: ', batch_ltokens.size(), index_bias.size())
                    indices = (batch_ltokens+index_bias)
                    
                    loss=neg_log_prob[indices]
#                     loss = self.cost_func(last_timestep_output, pseudo_labels)#labels[:,ii])
#                     losss.append(torch.unsqueeze(loss, dim=1))
#                 if is_train:
#                     loss = self.cost_func(last_timestep_output, max_idxs)#labels[:,0])
#                     losss.append(torch.unsqueeze(loss, dim=1))
            else:
                if is_train:
                    rand = random.random()
                    if rand<teaching_rate:
                        this_timestep_input = self.embed(labels[:,ii-1])#label teaching, lookup embedding
                    else:
                        this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                else:
                    this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                    
                last_timestep_hidden_state ,cx = self.lstmcell(this_timestep_input, (last_timestep_hidden_state,cx))
                last_timestep_output = self.fcnn(last_timestep_hidden_state)
                
                
                batch_ltokens = self.token_sample(logits=last_timestep_output, batch_size=batch_size, topk=topk, e_greedy_rate=e_greedy_rate, dec_seq_num=dec_seq_num)
                batch_ltokens=torch.LongTensor(batch_ltokens).view(-1)
                if self.use_cuda:
                    batch_ltokens = batch_ltokens.cuda()
                predicts.append(batch_ltokens.view(-1,1))
                if is_train:
                    neg_log_prob = -self.log_softmax(last_timestep_output).view(-1)
                    indices = (batch_ltokens+index_bias)
                    
                    loss+=neg_log_prob[indices]
                
        predicts = torch.cat(predicts, dim=1)
        #print('predicts size: ', predicts.size())
        
        if self.use_cuda:
            predicts = predicts.cpu().numpy()
        else:
            predicts = predicts.numpy()
        
        predicts_tokens=batch_tokens_remove_eos(predicts.tolist(), self.vocab)
        labels_tokens=[x for x in labels_tokens for ii in range(dec_seq_num)]
        labels_tokens=batch_tokens_remove_eos(labels_tokens, self.vocab)
        bleu_scores = batch_tokens_bleu(references=labels_tokens, candidates=predicts_tokens, smooth_epsilon=0.01)
#         predicts_words=batch_tokens2words(predicts_tokens, self.vocab)
#         predicts_sents=batch_words2sentence(predicts_words)
        labels_words = batch_tokens2words(labels_tokens, self.vocab)
        labels_sents = batch_words2sentence(labels_words)
#         print(len(labels_sents))
#         for ii in labels_sents[:21]:
#             print(ii)
        
        bleu_scores = torch.FloatTensor(bleu_scores).view(batch_size, dec_seq_num)
        bleu_mean = torch.mean(bleu_scores, dim=1).unsqueeze(dim=1)
        bleu_scores = bleu_scores-bleu_mean
        bleu_scores = bleu_scores.view(-1)
        
        bleu_scores = Variable(bleu_scores, requires_grad = 0)
        
        if self.use_cuda:
            bleu_scores = bleu_scores.cuda()
            
        loss = torch.dot(loss, bleu_scores)/batch_size/dec_seq_num
        
        return loss, predicts, torch.mean(bleu_mean.squeeze())
    
    def train_using_rl_3(self, enc_outputs, h0_and_c0, labels, topk=10):
        batch_size = enc_outputs.size(dim = 0)
        metadata = self.decode_by_beamsearch(encoder_hidden=h0_and_c0, encoder_outputs=enc_outputs, topk = topk)
        results = metadata['topk_sequence']
        results =torch.cat(results, dim = 2)
        results=results.view(batch_size*topk, -1)
        if self.use_cuda:
            results = results.data.cpu().tolist()
        else:
            results = results.data.tolist()
        results=batch_tokens_remove_eos(results, self.vocab)

        labels = [x for x in labels for ii in range(topk)]
        labels = batch_tokens_remove_eos(labels, self.vocab)
        bleu_scores = batch_tokens_bleu(references=labels, candidates=results, smooth_epsilon=0.01)
        
        bleu_scores = torch.FloatTensor(bleu_scores).view(batch_size, topk)
        bleu_max, _ = torch.max(bleu_scores, dim=1)
        
        bleu_mean = torch.mean(bleu_scores, dim=1).unsqueeze(dim=1)
        bleu_scores = bleu_scores-bleu_mean
        bleu_scores = bleu_scores.view(-1)
        
        bleu_scores = self._tocuda(Variable(bleu_scores, requires_grad = 0))
        
        log_probs = metadata['score']
        log_probs = log_probs.view(batch_size*topk)
        
        loss = -torch.dot(log_probs, bleu_scores)/batch_size/topk
        
        return loss, results, torch.mean(bleu_mean.squeeze()), torch.mean(bleu_max)
        
        
    def _tocuda(self, var):
        if self.use_cuda:
            return var.cuda()
        else:
            return var
    def decode_by_beamsearch(self, encoder_hidden=None, encoder_outputs=None, topk = 10):
        self.k = topk
        batch_size = encoder_outputs.size(dim=0)
        
        self.pos_index = self._tocuda(Variable(torch.LongTensor(range(batch_size)) * self.k).view(-1, 1))

        hidden = tuple([_inflate(h, self.k, 1).view(batch_size*self.k, -1) for h in encoder_hidden])
        #print('hidden0 size: (%s, %s)'%(hidden[0].size(), hidden[1].size()))

        # Initialize the scores; for the first step,
        # ignore the inflated copies to avoid duplicate entries in the top k
        sequence_scores = torch.Tensor(batch_size * self.k, 1)
        sequence_scores.fill_(-float('Inf'))
        sequence_scores.index_fill_(0, torch.LongTensor([i * self.k for i in range(0, batch_size)]), 0.0)
        sequence_scores = self._tocuda(Variable(sequence_scores))

        # Initialize the input vector
        input_var = self._tocuda(Variable(torch.LongTensor([self.SOS] * batch_size * self.k)))

        # Store decisions for backtracking
        stored_outputs = list()
        stored_scores = list()
        stored_predecessors = list()
        stored_emitted_symbols = list()
        stored_hidden = list()

        for ii in range(0, self.max_length):
            # Run the RNN one step forward
            #print('setp: %s'%ii)
            input_vec = self.embed(input_var)
            #print('input_var and input_vec size: ', input_var.size(), input_vec.size())
            hidden = self.lstmcell(input_vec, hidden)
            #print('hidden size: (%s, %s)'%(hidden[0].size(), hidden[1].size()))
            
            log_softmax_output = self.log_softmax(self.fcnn(hidden[0]))

            # To get the full sequence scores for the new candidates, add the local scores for t_i to the predecessor scores for t_(i-1)
            sequence_scores = _inflate(sequence_scores, self.V, 1)
            sequence_scores += log_softmax_output.squeeze(1)
            scores, candidates = sequence_scores.view(batch_size, -1).topk(self.k, dim=1)

            # Reshape input = (bk, 1) and sequence_scores = (bk, 1)
            input_var = (candidates % self.V).view(batch_size * self.k, 1)
            sequence_scores = scores.view(batch_size * self.k, 1)

            # Update fields for next timestep
            predecessors = (candidates / self.V + self.pos_index.expand_as(candidates)).view(batch_size * self.k, 1)
            if isinstance(hidden, tuple):
                hidden = tuple([h.index_select(0, predecessors.squeeze()) for h in hidden])
            else:
                hidden = hidden.index_select(0, predecessors.squeeze())

            # Update sequence scores and erase scores for end-of-sentence symbol so that they aren't expanded
            stored_scores.append(sequence_scores.clone())
            eos_indices = input_var.data.eq(self.EOS)
            if eos_indices.nonzero().dim() > 0:
                sequence_scores.data.masked_fill_(eos_indices, -float('inf'))

            # Cache results for backtracking
            stored_predecessors.append(predecessors)
            stored_emitted_symbols.append(input_var)
#             stored_hidden.append(hidden)

        # Do backtracking to return the optimal values
        output, h_t, h_n, s, l, p = self._backtrack(hidden,
                                                    stored_predecessors, stored_emitted_symbols,
                                                    stored_scores, batch_size, self.hidden_size)

        metadata = {}

        metadata['score'] = s
        metadata['topk_length'] = l
        metadata['topk_sequence'] = p
        metadata['length'] = [seq_len[0] for seq_len in l]
        metadata['sequence'] = [seq[0] for seq in p]
        
        torch.cuda.empty_cache()
        
        return metadata

    def _backtrack(self, hidden, predecessors, symbols, scores, b, hidden_size):
        """Backtracks over batch to generate optimal k-sequences.

        Args:
            nw_output [(batch*k, vocab_size)] * sequence_length: A Tensor of outputs from network
            nw_hidden [(num_layers, batch*k, hidden_size)] * sequence_length: A Tensor of hidden states from network
            predecessors [(batch*k)] * sequence_length: A Tensor of predecessors
            symbols [(batch*k)] * sequence_length: A Tensor of predicted tokens
            scores [(batch*k)] * sequence_length: A Tensor containing sequence scores for every token t = [0, ... , seq_len - 1]
            b: Size of the batch
            hidden_size: Size of the hidden state

        Returns:
            output [(batch, k, vocab_size)] * sequence_length: A list of the output probabilities (p_n)
            from the last layer of the RNN, for every n = [0, ... , seq_len - 1]

            h_t [(batch, k, hidden_size)] * sequence_length: A list containing the output features (h_n)
            from the last layer of the RNN, for every n = [0, ... , seq_len - 1]

            h_n(batch, k, hidden_size): A Tensor containing the last hidden state for all top-k sequences.

            score [batch, k]: A list containing the final scores for all top-k sequences

            length [batch, k]: A list specifying the length of each sequence in the top-k candidates

            p (batch, k, sequence_len): A Tensor containing predicted sequence
        """

        lstm = isinstance(hidden, tuple)

        # initialize return variables given different types
        output = list()
        h_t = list()
        p = list()
        # Placeholder for last hidden state of top-k sequences.
        # If a (top-k) sequence ends early in decoding, `h_n` contains
        # its hidden state when it sees EOS.  Otherwise, `h_n` contains
        # the last hidden state of decoding.
        if lstm:
            state_size = hidden[0].size()
            h_n = tuple([torch.zeros(state_size), torch.zeros(state_size)])
        else:
            h_n = torch.zeros(nw_hidden[0].size())
        l = [[self.max_length] * self.k for _ in range(b)]  # Placeholder for lengths of top-k sequences
                                                                # Similar to `h_n`

        # the last step output of the beams are not sorted
        # thus they are sorted here
        sorted_score, sorted_idx = scores[-1].view(b, self.k).topk(self.k)
        # initialize the sequence scores with the sorted last step beam scores
        s = sorted_score.clone()

        batch_eos_found = [0] * b   # the number of EOS found
                                    # in the backward loop below for each batch

        t = self.max_length - 1
        # initialize the back pointer with the sorted order of the last step beams.
        # add self.pos_index for indexing variable with b*k as the first dimension.
        t_predecessors = (sorted_idx + self.pos_index.expand_as(sorted_idx)).view(b * self.k)
        while t >= 0:
            # Re-order the variables with the back pointer
            current_symbol = symbols[t].index_select(0, t_predecessors)
            # Re-order the back pointer of the previous step with the back pointer of
            # the current step
            t_predecessors = predecessors[t].index_select(0, t_predecessors).squeeze()

            # This tricky block handles dropped sequences that see EOS earlier.
            # The basic idea is summarized below:
            #
            #   Terms:
            #       Ended sequences = sequences that see EOS early and dropped
            #       Survived sequences = sequences in the last step of the beams
            #
            #       Although the ended sequences are dropped during decoding,
            #   their generated symbols and complete backtracking information are still
            #   in the backtracking variables.
            #   For each batch, everytime we see an EOS in the backtracking process,
            #       1. If there is survived sequences in the return variables, replace
            #       the one with the lowest survived sequence score with the new ended
            #       sequences
            #       2. Otherwise, replace the ended sequence with the lowest sequence
            #       score with the new ended sequence
            #
            eos_indices = symbols[t].data.squeeze(1).eq(self.EOS).nonzero()
            if eos_indices.dim() > 0:
                for i in range(eos_indices.size(0)-1, -1, -1):
                    # Indices of the EOS symbol for both variables
                    # with b*k as the first dimension, and b, k for
                    # the first two dimensions
                    idx = eos_indices[i]
                    b_idx = int(idx[0] / self.k)
                    # The indices of the replacing position
                    # according to the replacement strategy noted above
                    res_k_idx = self.k - (batch_eos_found[b_idx] % self.k) - 1
                    batch_eos_found[b_idx] += 1
                    res_idx = b_idx * self.k + res_k_idx

                    # Replace the old information in return variables
                    # with the new ended sequence information
                    t_predecessors[res_idx] = predecessors[t][idx[0]]

                    current_symbol[res_idx, :] = symbols[t][idx[0]]
                    s[b_idx, res_k_idx] = scores[t][idx[0]]
                    l[b_idx][res_k_idx] = t + 1

            # record the back tracked results
            p.append(current_symbol)
            t -= 1

        # Sort and re-order again as the added ended sequences may change
        # the order (very unlikely)
        s, re_sorted_idx = s.topk(self.k)
        for b_idx in range(b):
            l[b_idx] = [l[b_idx][k_idx.data[0]] for k_idx in re_sorted_idx[b_idx,:]]

        re_sorted_idx = (re_sorted_idx + self.pos_index.expand_as(re_sorted_idx)).view(b * self.k)

        # Reverse the sequences and re-order at the same time
        # It is reversed because the backtracking happens in reverse time order
#         output = [step.index_select(0, re_sorted_idx).view(b, self.k, -1) for step in reversed(output)]
        p = [step.index_select(0, re_sorted_idx).view(b, self.k, -1) for step in reversed(p)]
        #    --- fake output ---
        output = None
        #    --- fake ---
        return output, h_t, h_n, s, l, p

    def _mask_symbol_scores(self, score, idx, masking_score=-float('inf')):
            score[idx] = masking_score

    def _mask(self, tensor, idx, dim=0, masking_score=-float('inf')):
        if len(idx.size()) > 0:
            indices = idx[:, 0]
            tensor.index_fill_(dim, indices, masking_score)

In [8]:
class Seq2Seq(nn.Module):
    def __init__(self, use_cuda, input_dim, hidden_dim, vocab, max_length = 25):
        super(Seq2Seq, self).__init__()
        
        self.use_cuda = use_cuda
        self.enc = Encoder(use_cuda=use_cuda, hidden_dim=hidden_dim, input_dim=input_dim, vocab=vocab)
        self.dec = Decoder(use_cuda=use_cuda, encoder=self.enc, hidden_dim=hidden_dim, max_length=max_length)
        if use_cuda:
            self.enc = self.enc.cuda()
            self.dec = self.dec.cuda()
    def forward(self, inputs, input_lens, labels, is_train=1, teaching_rate=1):
        enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
        if is_train:
            loss, predicts = self.dec(enc_outputs = enc_outputs, 
                                    h0_and_c0=(enc_hn, enc_cn), 
                                    sent_lens=input_lens,
                                    labels=torch.LongTensor(labels), 
                                    is_train=1, 
                                    teaching_rate = 1
                                    )
            return loss, predicts
        else:
            predicts = self.dec(enc_outputs = enc_outputs, 
                                h0_and_c0=(enc_hn, enc_cn), 
                                sent_lens=input_lens,
                                labels=torch.LongTensor(labels), 
                                is_train=0, 
                                teaching_rate = 1
                                )
            return predicts
#     def train_using_rl(self, inputs, input_lens, labels, is_train=1, teaching_rate=1):
#         enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
#         loss, predicts, bleu_mean = self.dec.train_using_rl_2(enc_outputs = enc_outputs, 
#                                                 h0_and_c0=(enc_hn, enc_cn), 
#                                                 sent_lens=input_lens,
#                                                 labels=labels,
#                                                 is_train=1, 
#                                                 teaching_rate = 1
#                                                 )
#         return loss, predicts, bleu_mean
    def train_using_rl(self, inputs, input_lens, labels, is_train=1, teaching_rate=1):
        topk=10
        enc_outputs, (enc_hn, enc_cn) = self.enc(torch.LongTensor(inputs), torch.LongTensor(input_lens))
        loss, results, bleu_mean, bleu_max = self.dec.train_using_rl_3(enc_outputs=enc_outputs, h0_and_c0=(enc_hn, enc_cn), labels=labels, topk=topk)
        return loss, results, bleu_mean, bleu_max

In [ ]:
# use_cuda = 1
# hidden_dim = 256
# input_dim = 100

# enc = Encoder(use_cuda=use_cuda, 
#             hidden_dim=hidden_dim, 
#             input_dim=input_dim, 
#             vocab=vocab
#            )
# if use_cuda:
#     enc = enc.cuda()
    
# sample_num = 11
# print('sentences length: ', train_set_input_lens[0:sample_num])

# enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(train_set_inputs[0:sample_num]), 
#                                     torch.LongTensor(train_set_input_lens[0:sample_num]))
# print('enc result size: ', enc_outputs.size(), enc_hn.size(), enc_cn.size())

# dec = Decoder(use_cuda=use_cuda, encoder=enc, hidden_dim=hidden_dim, max_length=61)
# if use_cuda:
#     dec = dec.cuda()
    
# loss, predicts = dec(enc_outputs = enc_outputs, 
#                     h0_and_c0=(enc_hn, enc_cn), 
#                     sent_lens=train_set_input_lens[0:sample_num], 
#                     labels=torch.LongTensor(train_set_labels[0:sample_num]), 
#                     is_train=1, teaching_rate = 1
#                     )
# print('loss is %4.7f'%loss.data[0])

# split_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, vocab = vocab, max_length = 61)
# a=time.time()
# loss, predicts = split_model.forward(torch.LongTensor(train_set_inputs[0:sample_num]), 
#                                      torch.LongTensor(train_set_input_lens[0:sample_num]), 
#                                      labels=torch.LongTensor(train_set_labels[0:sample_num]), 
#                                      is_train=1, teaching_rate=1)
# print('split_model: loss is %4.7f'%loss.data[0])
# print(time.time()-a)

# split_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, vocab = vocab, max_length = 61)
# a=time.time()
# loss, predicts, train_bleu_mean, train_bleu_max = split_model.train_using_rl(torch.LongTensor(train_set_inputs[0:sample_num]), 
#                                                                              torch.LongTensor(train_set_input_lens[0:sample_num]), 
#                                                                              labels=train_set_labels[0:sample_num], 
#                                                                              is_train=1, teaching_rate=1
#                                                                             )
# print('split_model: loss is %4.7f'%loss.data[0], train_bleu_mean)
# print(time.time()-a)

In [ ]:
use_cuda = 1
hidden_dim = 512
input_dim = 100
lr=0.005
batch_size=100
split_train_set_size=int(len(split_train_set_inputs)/1)
epochs=10000
train_bleu_mean=-1
train_bleu_max=-1
split_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 61)

fusion_model = Seq2Seq(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, 
                          vocab = vocab, max_length = 51)
#pre train para
# pre_train = torch.load('./models_saved/time-[2019-01-24-17-17-59]-info=[split_model]-loss=1.704468250-bleu=0.0485-hidden_dim=256-input_dim=100-epoch=9-batch_size=200-batch_id=[1-[of]-4949]-lr=0.0050', map_location='cpu')
# split_model.load_state_dict(pre_train)
# pre_train = torch.load('./models_saved/time-[2019-01-24-17-17-59]-info=[fusion_model]-loss=2.401571035-bleu=0.0334-hidden_dim=256-input_dim=100-epoch=9-batch_size=200-batch_id=[1-[of]-4949]-lr=0.0050', map_location='cpu')
# fusion_model.load_state_dict(pre_train)

if use_cuda:
    split_model = split_model.cuda()
    fusion_model = fusion_model.cuda()
    
split_optimizer = optim.Adam(filter(lambda p: p.requires_grad, split_model.parameters()), lr=lr)
fusion_optimizer = optim.Adam(filter(lambda p: p.requires_grad, fusion_model.parameters()), lr=lr)

start_time = time.time()

def model_train(epoch, batch_size, train_set_size):
    batch_id = 0
    valid_bleu = 0
    for start_idx in range(0, train_set_size-batch_size+1, batch_size):
#         print('batch id: ', batch_id)
            
        batch_id+=1
        end_idx = start_idx + batch_size
        
        split_optimizer.zero_grad()#clear
        fusion_optimizer.zero_grad()#clear
        
        split_loss, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[start_idx:end_idx]), 
                                     torch.LongTensor(split_train_set_input_lens[start_idx:end_idx]), 
                                     labels=torch.LongTensor(split_pseudo_train_set_labels[start_idx:end_idx]), 
                                     is_train=1, teaching_rate=1)
        
#         loss, predicts, train_bleu_mean, train_bleu_max = split_model.train_using_rl(torch.LongTensor(train_set_inputs[start_idx:end_idx]), 
#                                                                  torch.LongTensor(train_set_input_lens[start_idx:end_idx]), 
#                                                                  labels=train_set_labels[start_idx:end_idx], 
#                                                                  is_train=1, teaching_rate=1)
#         torch.cuda.empty_cache()
        #optimize
        split_loss.backward()#retain_graph=True)
        split_optimizer.step()
        
        fusion_loss, predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_train_set_inputs[start_idx:end_idx]), 
                                     torch.LongTensor(fusion_pseudo_train_set_input_lens[start_idx:end_idx]), 
                                     labels=torch.LongTensor(fusion_pseudo_train_set_labels[start_idx:end_idx]), 
                                     is_train=1, teaching_rate=1)
        fusion_loss.backward()#retain_graph=True)
        fusion_optimizer.step()

#         torch.cuda.empty_cache()
        
        if batch_id%50==1:
            split_model.eval()
            fusion_model.eval()
            
            sample_num = 5
            rand_idx = random.randint(0, train_set_size-sample_num-1)
            
            print('--------split model training sampling display--------')
            #teaching forcing
            loss_, predicts = split_model.forward(torch.LongTensor(split_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(split_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=torch.LongTensor(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num]), 
                                             is_train=1, teaching_rate=1)
            del loss_
            
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(split_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)
            
            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                
            print('--------fusion model training sampling display--------')
            loss_, predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_train_set_inputs[rand_idx:rand_idx+sample_num]), 
                                             torch.LongTensor(fusion_pseudo_train_set_input_lens[rand_idx:rand_idx+sample_num]), 
                                             labels=torch.LongTensor(fusion_pseudo_train_set_labels[rand_idx:rand_idx+sample_num]), 
                                             is_train=1, teaching_rate=1)
            del loss_
            
            predicts = batch_tokens_remove_eos(predicts, vocab)
            labels = batch_tokens_remove_eos(fusion_pseudo_train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
            predicts = batch_tokens2words(predicts, vocab)
            labels = batch_tokens2words(labels, vocab)
            
            predicts_sents = batch_words2sentence(predicts)
            labels_sents = batch_words2sentence(labels)
            
            for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
                print(' 1----> ', predict_sent)
                print(' 2----> ', label_sent)
                
#             #no teaching forcing
#             print('----no teaching forcing----')
#             predicts = split_model.forward(torch.LongTensor(train_set_inputs[rand_idx:rand_idx+sample_num]), 
#                                              torch.LongTensor(train_set_input_lens[rand_idx:rand_idx+sample_num]), 
#                                              labels=[],#torch.LongTensor(train_set_labels[rand_idx:rand_idx+sample_num]), 
#                                              is_train=0, teaching_rate=1)
#             predicts = batch_tokens_remove_eos(predicts, vocab)
#             labels = batch_tokens_remove_eos(train_set_labels[rand_idx:rand_idx+sample_num], vocab)
            
#             predicts = batch_tokens2words(predicts, vocab)
#             labels = batch_tokens2words(labels, vocab)
            
#             predicts_sents = batch_words2sentence(predicts)
#             labels_sents = batch_words2sentence(labels)

#             for (predict_sent, label_sent) in zip(predicts_sents, labels_sents):
#                 print(' 1----> ', predict_sent)
#                 print(' 2----> ', label_sent)
                
            info_stamp = 'split_loss={:2.9f}-train_bleu_mean={:2.9f}-train_bleu_max={:2.9f}-batch_size={:n}-epoch={:n}-batch_id=({:n}/{:n})'.format(
                              split_loss.data[0], train_bleu_mean, train_bleu_max, batch_size, epoch, batch_id, int(train_set_size/batch_size))
            print(info_stamp)
# #             #valid_set testing
            if batch_id%1000==1:
                rand_idx=random.randint(0, len(split_valid_set_inputs)-batch_size-1-1)
                predicts = split_model.forward(torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                 torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+batch_size]), 
                                                 labels=[],#torch.LongTensor(valid_set_labels[rand_idx:rand_idx+batch_size]), 
                                                 is_train=0, teaching_rate=1)
                predicts = batch_tokens_remove_eos(predicts, vocab)
                labels = batch_tokens_remove_eos(split_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], vocab)
                
                bleu_scores = batch_tokens_bleu(references=labels, candidates=predicts, smooth_epsilon=0.001)

                valid_bleu = 0
                for x in bleu_scores:
                    valid_bleu+=x
                valid_bleu/=len(bleu_scores)
                       
                info_stamp = 'info=[{:s}]-loss={:2.9f}-bleu={:1.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}'.format(
                              'split_model', split_loss.data[0], valid_bleu, hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr)
                print(info_stamp, valid_bleu)
                now = int(round(time.time()*1000))
                time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
                torch.save(split_model.state_dict(), ''.join(['./models_saved/', time_stamp, info_stamp]))


                rand_idx=random.randint(0, len(fusion_pseudo_valid_set_inputs)-batch_size-1-1)
                predicts = fusion_model.forward(torch.LongTensor(fusion_pseudo_valid_set_inputs[rand_idx:rand_idx+batch_size]), 
                                                 torch.LongTensor(fusion_pseudo_valid_set_input_lens[rand_idx:rand_idx+batch_size]), 
                                                 labels=[],#torch.LongTensor(valid_set_labels[rand_idx:rand_idx+batch_size]), 
                                                 is_train=0, teaching_rate=1)
                predicts = batch_tokens_remove_eos(predicts, vocab)
                labels = batch_tokens_remove_eos(fusion_pseudo_valid_set_labels[rand_idx:rand_idx+batch_size], vocab)
                
                bleu_scores = batch_tokens_bleu(references=labels, candidates=predicts, smooth_epsilon=0.001)

                valid_bleu = 0
                for x in bleu_scores:
                    valid_bleu+=x
                valid_bleu/=len(bleu_scores)
                info_stamp = 'info=[{:s}]-loss={:2.9f}-bleu={:1.4f}-hidden_dim={:n}-input_dim={:n}-epoch={:n}-batch_size={:n}-batch_id=[{:n}-[of]-{:n}]-lr={:1.4f}'.format(
                              'fusion_model', fusion_loss.data[0], valid_bleu, hidden_dim, input_dim, epoch, batch_size, batch_id, int(train_set_size/batch_size), lr)
                print(info_stamp, valid_bleu)
                now = int(round(time.time()*1000))
                time_stamp = time.strftime('time-[%Y-%m-%d-%H-%M-%S]-',time.localtime(now/1000))
                torch.save(fusion_model.state_dict(), ''.join(['./models_saved/', time_stamp, info_stamp]))
                
            split_model.train()
            fusion_model.train()
            
for epoch in range(epochs):
    model_train(epoch, batch_size, split_train_set_size)
    
print('running time: %.2f mins'%((time.time()-start_time)/60))

init lookup embedding matrix size:  torch.Size([44380, 100])
init lookup embedding matrix size:  torch.Size([44380, 100])
--------split model training sampling display--------
 1---->  ( <low_freq> . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 2---->  a right - handed middle - order batsman , lance <low_freq> made his . <split> first - class debut for south australia on his nineteenth birthday in 1941 .
 1---->  ( <low_freq> . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 2---->  a right - handed opening batsman who along with jack <low_freq> formed an opening pair in the 1930s regarded as one of the finest . <split> in test history , brown was a member of don bradman 's '' <low_freq> '' that toured england in 1948 without suffering a defeat .
 1---->  ( <low_freq> . . . . .
 2---->  a right - handed specialist batsman , he is the captain of ch

In [ ]:
print('running time: %.2f mins'%((time.time()-start_time)/60))



a=fusion_pseudo_train_set_labels[0]
print(len(a))